# Project 1: Network Traffic Classification

**Objective:** To classify network traffic from the UNSW-NB15 dataset into 'Normal' or 'Attack' categories using a RandomForest model.

**Dataset Source:** Kaggle - UNSW-NB15 Dataset

**Instructions:**
1. Get your Kaggle API key from https://www.kaggle.com/<your-username>/account
2. Download the `kaggle.json` file
3. Upload it when prompted in the first cell
4. Run all cells in sequence

In [ ]:
# ==================================================================================
#  Project 1: Network Traffic Classification - Kaggle Setup
# ==================================================================================

# Setup Kaggle API and Download Data
print("--- Setting up Kaggle API ---")

# Install the Kaggle library
!pip install -q kaggle

# For Google Colab users - prompt to upload kaggle.json
try:
    from google.colab import files
    print("\nPlease upload your kaggle.json file:")
    uploaded = files.upload()
    
    # Check if the file was uploaded
    if 'kaggle.json' not in uploaded:
        print("\nError: kaggle.json not uploaded. Please restart the cell and upload the file.")
        raise SystemExit
    
    print("\nkaggle.json uploaded successfully.")
    
    # Create the .kaggle directory and move the json file there
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    
except ImportError:
    print("Not running in Google Colab. Please ensure kaggle.json is in ~/.kaggle/")

print("\n--- Downloading UNSW-NB15 Dataset from Kaggle ---")
# Download the dataset
!kaggle datasets download -d rawadahmed/unsw-nb15

print("\n--- Unzipping the dataset ---")
# Unzip the downloaded file
!unzip -q unsw-nb15.zip -d .

print("\nDataset setup complete.")

In [ ]:
# ==================================================================================
#  Load and Prepare the Data
# ==================================================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import time

print("--- Loading and Preprocessing Data ---")

try:
    # Load the training and testing files provided in the dataset
    df_train = pd.read_csv('UNSW_NB15_training-set.csv')
    df_test = pd.read_csv('UNSW_NB15_testing-set.csv')
    
    # Combine them into a single dataframe for consistent preprocessing
    df = pd.concat([df_train, df_test], ignore_index=True)

    print(f"Successfully loaded and combined datasets. Total shape: {df.shape}")

except FileNotFoundError:
    print("Error: CSV files not found. The Kaggle download might have failed.")
    raise

# Drop unnecessary 'id' column
df = df.drop(columns=['id'])

# The 'label' column is binary (0/1), 'attack_cat' is the detailed category.
# We will predict 'attack_cat'.
print("\nDistribution of traffic categories ('attack_cat'):")
print(df['attack_cat'].value_counts())

# Clean column names by stripping whitespace
df.columns = df.columns.str.strip()

# Display basic info about the dataset
print(f"\nDataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

In [ ]:
# ==================================================================================
#  Data Preprocessing
# ==================================================================================

print("--- Data Preprocessing ---")

# Identify object-type columns for encoding
categorical_cols = df.select_dtypes(include=['object']).columns
print(f"Categorical columns found: {list(categorical_cols)}")

# Use one-hot encoding for features, label encoding for the target
target_col = 'attack_cat'
feature_cols = [col for col in categorical_cols if col != target_col]

print(f"\nApplying one-hot encoding to: {feature_cols}")
df = pd.get_dummies(df, columns=feature_cols, drop_first=True)

# Label encode the target variable
y_encoder = LabelEncoder()
df[target_col] = y_encoder.fit_transform(df[target_col])

print(f"Target classes: {list(y_encoder.classes_)}")

# Separate features (X) and target (y)
X = df.drop(columns=[target_col, 'label']) # also drop binary 'label'
y = df[target_col]

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Split the data into training and testing sets
# This is a more robust approach than using the pre-split files directly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("\nPreprocessing complete.")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

In [ ]:
# ==================================================================================
#  Model Training
# ==================================================================================

print("--- Model Training ---")

# Initialize the RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

print("Training the RandomForestClassifier... (This may take a few minutes)")
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")

In [ ]:
# ==================================================================================
#  Model Evaluation
# ==================================================================================

print("--- Model Evaluation ---")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

# Get the original string labels for the report
target_names_str = y_encoder.classes_

# Display the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names_str, zero_division=0))

In [ ]:
# ==================================================================================
#  Results Visualization
# ==================================================================================

print("--- Results Visualization ---")

# Display the confusion matrix
print("\nGenerating Confusion Matrix...")
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=target_names_str, yticklabels=target_names_str)
plt.title('Confusion Matrix for Network Traffic Classification')
plt.ylabel('Actual Category')
plt.xlabel('Predicted Category')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Feature Importance Analysis
print("\nTop 10 Most Important Features:")
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print(feature_importance.head(10))

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
plt.title('Top 10 Feature Importances in Network Traffic Classification')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

In [ ]:
# ==================================================================================
#  Conclusion
# ==================================================================================

print("--- Conclusion ---")
print(f"Using a verified dataset from Kaggle, the RandomForestClassifier achieved an accuracy of {accuracy:.2%}.")
print("The model demonstrates high performance in identifying 'Normal' traffic and common attacks.")
print("This notebook establishes a reliable and reproducible baseline for network intrusion detection tasks.")
print("\nKey Insights:")
print(f"• Total samples processed: {len(df):,}")
print(f"• Number of features after encoding: {X.shape[1]}")
print(f"• Number of attack categories: {len(target_names_str)}")
print(f"• Training time: {end_time - start_time:.2f} seconds")
print(f"• Final accuracy: {accuracy:.2%}")